In [1]:
"""
Code for section 7. Word Similarity and Analogy
"""
import os
import torch
from torch import nn

In [5]:
class TokenEmbedding:
    """Token Embedding."""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = "../../Data/" + embedding_name
        # GloVe website: https://nlp.stanford.edu/projects/glove/
        # fastText website: https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r', encoding='utf-8') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # Skip header information, such as the top row in fastText
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [6]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

In [8]:
def knn(W, x, k):
    """
    k-nearest neighbors -> get word similarity
        Add 1e-9 for numerical stability
    """
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    vals, topk = torch.topk(cos, k=k)
    return topk, vals

In [9]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # Exclude the input word
        print(f'cosine sim={float(c):.3f}: {embed.idx_to_token[int(i)]}')

In [10]:
get_similar_tokens('chip', 10, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics
cosine sim=0.731: semiconductor
cosine sim=0.716: maker
cosine sim=0.708: computer
cosine sim=0.707: microprocessor
cosine sim=0.703: makers
cosine sim=0.701: micro
cosine sim=0.691: manufacturing


In [11]:
get_similar_tokens('love', 10, glove_6b50d)

cosine sim=0.843: dream
cosine sim=0.840: life
cosine sim=0.840: dreams
cosine sim=0.836: loves
cosine sim=0.835: me
cosine sim=0.823: my
cosine sim=0.822: mind
cosine sim=0.811: loving
cosine sim=0.807: wonder
cosine sim=0.801: soul


In [12]:
get_similar_tokens('mother', 10, glove_6b50d)

cosine sim=0.946: daughter
cosine sim=0.943: wife
cosine sim=0.934: grandmother
cosine sim=0.906: husband
cosine sim=0.891: father
cosine sim=0.891: sister
cosine sim=0.879: her
cosine sim=0.876: woman
cosine sim=0.876: aunt
cosine sim=0.860: friend


In [13]:
def get_analogy(token_a, token_b, token_c, embed):
    """
    get word analogy
    """
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[int(topk[0])]  # Remove unknown words

In [14]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [15]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

In [16]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

In [17]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'